In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sb
plt.rcParams['figure.figsize'] = (20, 12)

# ETL (extract, transform, load)

## load csv, shp file

In [4]:
#load into main df_csv
df_csv = pd.read_csv('rm_crop_yields_1938_2021.csv')

In [5]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25017 entries, 0 to 25016
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          25017 non-null  int64  
 1   RM            25017 non-null  int64  
 2   Winter Wheat  3037 non-null   float64
 3   Canola        14008 non-null  float64
 4   Spring Wheat  24924 non-null  float64
 5   Mustard       4487 non-null   float64
 6   Durum         11581 non-null  float64
 7   Sunflowers    946 non-null    float64
 8   Oats          23913 non-null  float64
 9   Lentils       5515 non-null   float64
 10  Peas          8134 non-null   float64
 11  Barley        24703 non-null  float64
 12  Fall Rye      15847 non-null  float64
 13  Canary Seed   3819 non-null   float64
 14  Spring Rye    805 non-null    float64
 15  Tame Hay      4205 non-null   float64
 16  Flax          20934 non-null  float64
 17  Chickpeas     960 non-null    float64
dtypes: float64(16), int64(2)
m

In [6]:
df_csv.rename(columns={"Winter Wheat": "WinterWheat", "Spring Wheat": "SpringWheat","Fall Rye":"FallRye",\
             "Canary Seed":"CanarySeed","Spring Rye":"SpringRye","Tame Hay":"TameHay"},inplace=True)

In [7]:
#https://saskpulse.com
#https://www.rayglen.com/grain-conversion-calculator/
# (lbs/ac) -> Mustard (50 lbs/bu), Sunflower (30 lbs/bu), Lentils (60 lbs/bu), 
#             Canary Seeed (50 lbs/bu), Chickpeas (60lb/bu)

# Tame Hay (tons/ac)
# All the rest are bushel/acre (bu/ac)
df_csv['Mustard']=df_csv['Mustard']/50
df_csv['Sunflowers']=df_csv['Sunflowers']/30
df_csv['Lentils']=df_csv['Lentils']/60
df_csv['CanarySeed']=df_csv['CanarySeed']/50
df_csv['Chickpeas']=df_csv['Chickpeas']/60

In [8]:
#load shp data
gdf = gpd.read_file('Rural Municipality.shp')

#drop columns that won't be using
gdf.drop(['PPID','EFFDT','EXPDT','FEATURECD','SHAPE_AREA','SHAPE_LEN'],axis=1,inplace=True)

#rename column to match with main df
gdf.rename(columns=
{   'RMNO': 'RM',
    'RMNM': 'Municipality'
}, inplace=True)

#match data type between df_csv and gdf
gdf['RM']=gdf['RM'].astype('int')
gdf['Municipality']=gdf['Municipality'].astype('string')


In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   RM            298 non-null    int64   
 1   Municipality  298 non-null    string  
 2   geometry      298 non-null    geometry
dtypes: geometry(1), int64(1), string(1)
memory usage: 7.1 KB


# EDA (Extrapolatory Data Analysis)

#### check for unique values

In [10]:
gdf['RM'].unique()
gdf['RM'].nunique()
#298 unique RM

298

#### check for duplicated values

In [11]:
gdf.duplicated().sum()

0

In [12]:
df_describe = df_csv.describe().copy()
df_describe
# Total of 25017 rows
# Year from 1938 to 2021 ~ 84 years
# 299 RM from 1 to 622 
# Spring Wheat, Barley and Oats have the most rows -> more complete data?
# Oats, Winter Wheat, Barley has the most mean -> most yield
# Tame Hay, Spring Rye, Flax have the least mean -> least yield

,Year,RM,WinterWheat,Canola,SpringWheat,Mustard,Durum,Sunflowers,Oats,Lentils,Peas,Barley,FallRye,CanarySeed,SpringRye,TameHay,Flax,Chickpeas
count,25017.000000,25017.000000,3037.00000,14008.000000,24924.000000,4487.000000,11581.000000,946.000000,23913.000000,5515.000000,8134.000000,24703.000000,15847.000000,3819.000000,805.000000,4205.000000,20934.000000,960.000000
mean,1979.385938,248.957509,38.24350,24.146796,25.778733,16.883719,29.024429,23.087664,47.104433,19.529647,30.971982,38.022104,21.225298,19.902804,7.575776,1.181520,14.287663,23.467677
std,24.221744,148.829628,12.20715,9.209177,11.153028,5.514007,9.829962,18.525887,21.775369,6.815627,10.223101,16.610425,10.101176,7.084164,12.473814,0.654725,7.082308,9.660584
min,1938.000000,1.000000,3.00000,0.000000,0.000000,0.712000,0.000000,0.000000,0.000000,0.766667,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.516667
25%,1958.000000,124.000000,30.00000,18.000000,19.000000,13.270000,23.000000,0.000000,33.000000,15.000000,24.300000,26.000000,15.000000,15.140000,0.000000,0.750000,9.000000,16.666667
50%,1979.000000,247.000000,38.00000,23.000000,25.600000,16.940000,28.200000,26.350000,46.000000,19.433333,31.000000,37.900000,20.000000,20.000000,0.000000,1.200000,14.000000,22.941667
75%,2000.000000,370.000000,45.50000,29.200000,31.800000,20.018900,34.300000,36.666667,59.000000,23.983333,37.900000,48.750000,26.300000,24.000000,20.000000,1.570000,19.200000,29.704167
max,2021.000000,622.000000,90.00000,59.500000,198.000000,50.000000,85.370000,80.000000,165.100000,56.850000,86.800000,105.940000,108.000000,54.540000,46.500000,4.000000,110.000000,90.000000


In [13]:
#count unique RM
df_csv['RM'].unique()
df_csv['RM'].nunique()

299

In [14]:
#check for duplicated rows
df_csv.duplicated().sum()

0

In [15]:
#check how many records there are each year
#-->not all have data for all years
temp_df= df_csv.groupby('Year').count()['RM']
temp_df = temp_df[temp_df!=299]
temp_df

Year
1991    298
1992    297
1993    297
1994    297
1995    297
1996    297
1997    297
1998    297
1999    296
2000    296
2001    296
2002    296
2003    296
2004    296
2005    296
2006    296
2007    296
2008    296
2009    293
2010    295
2011    295
2012    295
2013    295
2014    295
2015    295
2016    295
2017    295
2018    295
2019    295
2020    295
2021    295
Name: RM, dtype: int64

In [16]:
#check for RM with less than 84 years of data
temp_df= df_csv.groupby('RM').count()['Year']
temp_df = temp_df[temp_df!=84]
temp_df

RM
132    83
278    71
408    61
521    53
529    54
555    83
Name: Year, dtype: int64

In [17]:
#Municipality with less than 84 years of data

pd.merge(temp_df,gdf,on='RM')

,RM,Year,Municipality,geometry
0,132,83,HILLSBOROUGH,"POLYGON ((445175.620 5573313.600, 445572.880 5..."
1,521,53,DISTRICT OF LAKELAND,"MULTIPOLYGON (((442578.660 5974009.340, 442566..."
2,555,83,BIG RIVER,"POLYGON ((382623.457 5977383.405, 382824.163 5..."


In [18]:
#check for RM not in geodata

temp_df= df_csv.groupby('RM').count().index
temp_df = temp_df[~temp_df.isin(gdf['RM'])]
temp_df

#278 Kutawa, Prairie No. 408, Greenfield No. 529

Int64Index([278, 408, 529], dtype='int64', name='RM')

In [19]:
#check for geodata RM not in main data set

temp_df= df_csv.groupby('RM').count().index
temp_gdf = gdf[~gdf['RM'].isin(temp_df)]
temp_gdf

,RM,Municipality,geometry
179,999,NORTHERN ADMIN DISTRICT,"MULTIPOLYGON (((666659.844 6655115.377, 666876..."
201,998,NORTHERN ADMIN DISTRICT,"MULTIPOLYGON (((443384.000 6596155.000, 442966..."
